In [1]:
from pyhive import hive
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
from sqlalchemy import create_engine

In [2]:
conn = hive.Connection(host='localhost', port=10000, username='rodrigo', database='youtube')
engine = create_engine('hive://', creator=lambda: conn)

In [32]:
df_result = pd.read_sql_query(
    ' SELECT * '
    ' FROM TOTAL_VISUALIZACOES_POR_SEMANA '
    ' where TOTAL_VISUALIZACOES_POR_SEMANA.assunto = "assunto_cities_skylines" '
    ' AND TOTAL_VISUALIZACOES_POR_SEMANA.data_extracao = "2023-10-27"', 
    con=engine
)

In [34]:
df_result.head()

,nm_canal,titulo_video,total_caractere_video,tags,duracao_video_minutos,total_tags,total_visualizacoes,total_comentarios,total_likes,total_visualizacoes_dia,total_comentarios_dia,total_likes_dia,assunto,data_extracao,id_canal,id_video
0,JOGA COM WILLIAM,"CITIES SKYLINE 2 - CONSTRUIR PARQUES, BOMBEIRO...",92,"[""joga com william"",""jogos"",""jogos de corrida""...",0.55,24,4,0,2.0,4,0,2.0,assunto_cities_skylines,2023-10-27,UC-N8y8IuT8B44h-IQQAmq4w,UpwkXP1Hjy8
1,Irmãos Cities,"Cities Skylines, ÚLTIMAS PRAIAS!! - Mountain V...",70,[],30.55,0,481,19,69.0,1,19,69.0,assunto_cities_skylines,2023-10-27,UC1mk6EtfMjxR4eEZ7C43zTQ,2c5QZgVUdGo
2,Irmãos Cities,"Cities Skylines, contrução do ESTÁDIO DE BASEB...",88,[],27.00,0,342,11,52.0,342,11,52.0,assunto_cities_skylines,2023-10-27,UC1mk6EtfMjxR4eEZ7C43zTQ,6qHqkq9yZpM
3,Irmãos Cities,"Cities Skylines, contrução do PARQUE DE DIVERS...",89,[],26.33,0,481,19,86.0,481,19,86.0,assunto_cities_skylines,2023-10-27,UC1mk6EtfMjxR4eEZ7C43zTQ,7qO2gNyBnLM
4,Irmãos Cities,"Cities Skylines, CIDADE VISTA DE NOITE!! - Mou...",77,[],27.42,0,400,20,58.0,1,20,58.0,assunto_cities_skylines,2023-10-27,UC1mk6EtfMjxR4eEZ7C43zTQ,9aTPAbE-DP8


In [35]:
df_result.dtypes

nm_canal                    object
titulo_video                object
total_caractere_video        int64
tags                        object
duracao_video_minutos      float64
total_tags                   int64
total_visualizacoes          int64
total_comentarios            int64
total_likes                float64
total_visualizacoes_dia      int64
total_comentarios_dia        int64
total_likes_dia            float64
assunto                     object
data_extracao               object
id_canal                    object
id_video                    object
dtype: object

In [36]:
df_result.shape

(245, 16)

# Relação Comentários X Visualização (proporção) e Relação Visualizações X Likes (proporção)

In [37]:

fig = px.scatter(df_result, x='total_caractere_video', y='total_visualizacoes')
df_result.sort_values(by='total_visualizacoes', ascending=False).head()


,nm_canal,titulo_video,total_caractere_video,tags,duracao_video_minutos,total_tags,total_visualizacoes,total_comentarios,total_likes,total_visualizacoes_dia,total_comentarios_dia,total_likes_dia,assunto,data_extracao,id_canal,id_video
213,ChratosGameplay,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,75,"[""cities skylines"",""cities skylines 2"",""cities...",19.63,31,120812,418,13988.0,3437,2,204.0,assunto_cities_skylines,2023-10-27,UCrOH1V-FyMunBIMrKL0y0xQ,wCLSZxLfUAk
210,ChratosGameplay,PEQUENA FORTUNA EM SERVIÇOS PÚBLICOS BÁSICOS 🏙...,70,"[""cities skylines"",""cities skylines 2"",""cities...",31.92,31,71357,188,8978.0,1889,2,139.0,assunto_cities_skylines,2023-10-27,UCrOH1V-FyMunBIMrKL0y0xQ,totHMIv6_hI
212,ChratosGameplay,PRIMEIRA PRAÇA DA CIDADE SAI DO PAPEL 🏙️ - Cit...,63,"[""cities skylines"",""cities skylines 2"",""cities...",22.95,31,60786,201,7321.0,1568,1,156.0,assunto_cities_skylines,2023-10-27,UCrOH1V-FyMunBIMrKL0y0xQ,vy-upTpchig
202,ChratosGameplay,SUPER SUBÚRBIO FAZ ALCANÇAR 10000 HABITANTES 🏙...,70,"[""cities skylines"",""cities skylines 2"",""cities...",25.67,31,54375,213,6855.0,1561,213,148.0,assunto_cities_skylines,2023-10-27,UCrOH1V-FyMunBIMrKL0y0xQ,9SGBZ7GE5EA
203,ChratosGameplay,POLUIÇÃO FORÇA MIGRAÇÃO DA INDÚSTRIA PARA NOVO...,81,"[""cities skylines"",""cities skylines 2"",""cities...",28.43,31,50789,124,6448.0,1853,124,165.0,assunto_cities_skylines,2023-10-27,UCrOH1V-FyMunBIMrKL0y0xQ,KfzZNkdyeEs


In [38]:
fig.show()

- Comparação Likes X Visualizações e Cometários X Visualizações

In [40]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['Likes X Visualizações', 'Cometários X Visualizações', 'TOTAL_CARACTERE_VIDEO X VISUALIZAÇÔES'])
sc1 = px.scatter(df_result, x='total_likes', y='total_visualizacoes',)
fig.add_trace(sc1['data'][0], row=1, col=1)
fig.update_xaxes(title_text='Likes', row=1, col=1)
fig.update_yaxes(title_text='Visualizações', row=1, col=1)

sc2 = px.scatter(df_result, x='total_comentarios', y='total_visualizacoes')
fig.add_trace(sc2['data'][0], row=1, col=2)
fig.update_xaxes(title_text='Comentários', row=1, col=2)
fig.update_yaxes(title_text='Visualizações', row=1, col=2)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)

sc3 = px.scatter(df_result, x='total_caractere_video', y='total_visualizacoes')
fig.add_trace(sc3['data'][0], row=1, col=3)
fig.update_xaxes(title_text='TOTAL_CARACTERE_VIDEO', row=1, col=3)
fig.update_yaxes(title_text='Visualizações', row=1, col=3)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)



fig.show()

# Dia em que os vídeos são Públicados com alta frequência

In [10]:
df_publicacao_video = pd.read_sql_query(
    'SELECT semana_traduzida, '
    ' nm_canal,  '
    ' total_videos, '
    ' data_publicacao,  '
    ' assunto, '
    ' id_canal '
    ' FROM youtube.total_video_publicado_semana'
    ' WHERE assunto = "assunto_cities_skylines" ',
    con=engine
)

In [11]:
def indice_semana(dia: str) -> int: 
    dias_semana = {
        'Domingo': 1,
        'Segunda-feira' : 2,
        'Terça-feira' : 3,
        'Quarta-feira' : 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado' : 7
    }

    return dias_semana.get(dia)


In [12]:
df_publicacao_video.head()

,semana_traduzida,nm_canal,total_videos,data_publicacao,assunto,id_canal
0,Domingo,Irmãos Cities,1,2023-10-15,assunto_cities_skylines,UC1mk6EtfMjxR4eEZ7C43zTQ
1,Domingo,CANAL HAND - GAME WITH MUSIC,1,2023-10-15,assunto_cities_skylines,UC7v51W2NoLFawizgcNuVBOA
2,Domingo,GAC TV,1,2023-10-15,assunto_cities_skylines,UCEc5jA0NvvR-yJC0SISV6nw
3,Domingo,Lucas gameplays brasil,1,2023-10-15,assunto_cities_skylines,UCOYJm7BsLV8VCkW2DC1MI5w
4,Domingo,O Veio Games,1,2023-10-15,assunto_cities_skylines,UCOe_x5ararWwWajBKbFKNkA


In [13]:

# df_publicacao_video['indice_semana'] = df_publicacao_video['semana_traduzida'].apply(indice_semana)
# df_publicacao_video.sort_values(by='indice_semana', inplace=True)
df_publicacao_video = df_publicacao_video.groupby('semana_traduzida').sum('total_videos').reset_index()
df_publicacao_video['indice_semana'] = df_publicacao_video['semana_traduzida'].apply(indice_semana)
df_publicacao_video.sort_values(by='indice_semana', inplace=True)
df_publicacao_video.drop(['indice_semana'], axis=1, inplace=True)
df_publicacao_video.head(7)

,semana_traduzida,total_videos
0,Domingo,25
3,Segunda-feira,40
6,Terça-feira,39
1,Quarta-feira,37
2,Quinta-feira,39
4,Sexta-feira,34
5,Sábado,21


In [14]:
fig = px.bar(df_publicacao_video, x='semana_traduzida',y='total_videos', text_auto=True)
fig.update_layout(title_text='Envio de Vídeo por Semana', showlegend=True, title=dict(x=0.5) )

fig.show()

# Total de Tags X Vísualizações 

In [25]:
df_tags_visualizacoes = pd.read_sql_query(' SELECT  '
        '   id_canal,    ' 
        '	total_tags , '
        '	total_visualizacoes ' 
        ' FROM total_visualizacoes_por_semana tvps  '
        ' WHERE tvps.assunto  = "assunto_cities_skylines" '
        ' AND data_extracao = "2023-10-27" ',
    con=engine
)

In [26]:
df_tags_visualizacoes.head()

,id_canal,total_tags,total_visualizacoes
0,UC-N8y8IuT8B44h-IQQAmq4w,24,4
1,UC1mk6EtfMjxR4eEZ7C43zTQ,0,481
2,UC1mk6EtfMjxR4eEZ7C43zTQ,0,342
3,UC1mk6EtfMjxR4eEZ7C43zTQ,0,481
4,UC1mk6EtfMjxR4eEZ7C43zTQ,0,400


In [27]:
df_tags_visualizacoes.drop_duplicates(inplace=True)

In [28]:
df_tags_visualizacoes.head()

,id_canal,total_tags,total_visualizacoes
0,UC-N8y8IuT8B44h-IQQAmq4w,24,4
1,UC1mk6EtfMjxR4eEZ7C43zTQ,0,481
2,UC1mk6EtfMjxR4eEZ7C43zTQ,0,342
4,UC1mk6EtfMjxR4eEZ7C43zTQ,0,400
5,UC1mk6EtfMjxR4eEZ7C43zTQ,0,537


In [29]:
fig = px.scatter(df_tags_visualizacoes, x='total_tags', y='total_visualizacoes')
fig.show()

# Total Víews DIA por Vídeo

In [60]:
df_views_video = pd.read_sql_query(
    ' SELECT titulo_video, '
    ' total_visualizacoes, '
    ' total_visualizacoes_dia '
    ' FROM total_visualizacoes_por_semana tvps '
    ' where id_canal  = "UCrOH1V-FyMunBIMrKL0y0xQ" '
    ' AND data_extracao = "2023-10-20" '
    ' AND total_visualizacoes  > 0 ',
    con=engine
)
df_views_video.head() 

,titulo_video,total_visualizacoes,total_visualizacoes_dia
0,MODDERS decidem fazer O MELHOR JOGO DE GERENCI...,9112,163
1,Planejamento inicial PODE SALVAR SUA CIDADE! 🚗...,16805,353
2,PEQUENA FORTUNA EM SERVIÇOS PÚBLICOS BÁSICOS 🏙...,12838,12838
3,"DO CRIADOR DA MELHOR CIDADE JÁ FEITA, A NOVA M...",22599,2274
4,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,52156,34195


In [61]:
df_views_video.shape

(12, 3)

In [62]:
df_views_video['titulo_video'] = df_views_video['titulo_video'].apply(lambda x: x.capitalize())
df_views_video.drop_duplicates(inplace=True)
df_views_video.sort_values(by='total_visualizacoes_dia', ascending=True, inplace=True)
fig = px.bar(df_views_video, x='total_visualizacoes_dia', y='titulo_video', orientation='h', text_auto=True)
fig.show()


- Selecionando Vídeo - (visualização, vídeo, comentário, likes)

In [76]:
df_desempenho_video = pd.read_sql_query(
    ' SELECT data_extracao, '
    ' total_visualizacoes_dia '
    ' FROM total_visualizacoes_por_semana tvps '
    ' where tvps.assunto  = "assunto_cities_skylines" '
    ' AND tvps.id_video = "wCLSZxLfUAk" ' ,
    con=engine
)
df_desempenho_video.head()

,data_extracao,total_visualizacoes_dia
0,2023-10-19,17961
1,2023-10-20,34195
2,2023-10-21,23882
3,2023-10-22,12800
4,2023-10-23,8201


In [77]:
df_desempenho_video['data_extracao'] = pd.to_datetime(df_desempenho_video['data_extracao'])
df_desempenho_video.dtypes

data_extracao              datetime64[ns]
total_visualizacoes_dia             int64
dtype: object

In [80]:
fig = px.bar(data_frame=df_desempenho_video, x='data_extracao', y='total_visualizacoes_dia')
fig.show()

# Total Víews DIA por Canal

In [97]:
df_views_canal = pd.read_sql_query(
    ' SELECT '
    '    nm_canal, '
    '    total_visualizacoes_dia '
    '    FROM total_visualizacoes_por_semana  '
    '    WHERE data_extracao="2023-10-27"  '
    '    AND ASSUNTO = "assunto_cities_skylines" '
    '    AND total_visualizacoes_dia > 0  ',
    con=engine
)

df_views_canal['nm_canal'] = df_views_canal['nm_canal'].apply(lambda x: x.capitalize())
df_views_canal.head()

,nm_canal,total_visualizacoes_dia
0,Joga com william,4
1,Irmãos cities,1
2,Irmãos cities,342
3,Irmãos cities,481
4,Irmãos cities,1


In [98]:
df_views_canal = df_views_canal.groupby('nm_canal').sum('total_visualizacoes_dia').sort_values(by='total_visualizacoes_dia', ascending=True).reset_index()

In [99]:
df_views_canal.dtypes

nm_canal                   object
total_visualizacoes_dia     int64
dtype: object

In [101]:
df_views_canal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   nm_canal                 102 non-null    object
 1   total_visualizacoes_dia  102 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [102]:
df_views_canal['nm_canal'] = df_views_canal['nm_canal'].astype('string')

In [103]:
df_views_canal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   nm_canal                 102 non-null    string
 1   total_visualizacoes_dia  102 non-null    int64 
dtypes: int64(1), string(1)
memory usage: 1.7 KB


In [100]:

df_views_canal.head()
fig = px.bar(df_views_canal, x='total_visualizacoes_dia', y='nm_canal', orientation='h', text_auto=True)
fig.show() 

# A duração do vídeo é importante para visualizações e interação (curtidas/comentários)?

In [125]:
df_duracao_video = pd.read_sql_query(
    ' SELECT DURACAO_VIDEO_MINUTOS, TOTAL_COMENTARIOS, TOTAL_LIKES, TOTAL_VISUALIZACOES'
    ' FROM total_visualizacoes_por_semana '
    ' WHERE ID_CANAL IN ("UCrOH1V-FyMunBIMrKL0y0xQ") '
    '    AND data_extracao="2023-10-27"  ',
    con=engine
    )
df_duracao_video.head()

,duracao_video_minutos,total_comentarios,total_likes,total_visualizacoes
0,0.00,0,50,0
1,26.88,220,5600,42650
2,25.67,213,6855,54375
3,28.43,124,6448,50789
4,32.28,180,5046,39265


In [126]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['DURACAO_VIDEO_MINUTOS X TOTAL_LIKES', 'DURACAO_VIDEO_MINUTOS X TOTAL_COMENTARIOS', 'DURACAO_VIDEO_MINUTOS X TOTAL_VISUALIZACOES'])
sc1 = px.scatter(df_duracao_video, x='duracao_video_minutos', y='total_likes',)
fig.add_trace(sc1['data'][0], row=1, col=1)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=1)
fig.update_yaxes(title_text='TOTAL_LIKES', row=1, col=1)

sc2 = px.scatter(df_duracao_video, x='duracao_video_minutos', y='total_comentarios')
fig.add_trace(sc2['data'][0], row=1, col=2)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=2)
fig.update_yaxes(title_text='TOTAL_COMENTARIOS', row=1, col=2)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)

sc3 = px.scatter(df_duracao_video, x='duracao_video_minutos', y='total_visualizacoes')
fig.add_trace(sc3['data'][0], row=1, col=3)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=3)
fig.update_yaxes(title_text='TOTAL_VISUALIZACOES', row=1, col=3)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)


# Desepenho dos Canais (like, comentários, vísualizações) Faça Comparação


In [19]:
df_duracao_video = pd.read_sql_query(
    ' SELECT data_extracao, NM_CANAL, TOTAL_VISUALIZACOES_DIA, TOTAL_LIKES_DIA  , TOTAL_COMENTARIOS_DIA'
    ' FROM total_visualizacoes_por_semana '
    ' WHERE  ID_CANAL IN ("UCrOH1V-FyMunBIMrKL0y0xQ" , "UCe9jrI0YQ5SM6h5QRZ9FZlA")  ',

    con=engine
    )

df_duracao_video.head()

,data_extracao,nm_canal,total_visualizacoes_dia,total_likes_dia,total_comentarios_dia
0,2023-10-15,ChratosGameplay,4562,687,23
1,2023-10-16,ChratosGameplay,2837,236,6
2,2023-10-16,ChratosGameplay,7914,1517,55
3,2023-10-19,Barbosa Cities,5011,795,48
4,2023-10-19,Barbosa Cities,7819,2011,120


In [20]:
df_duracao_video = df_duracao_video.groupby(['data_extracao', 'nm_canal']) \
    .agg(
        total__visualizacoes=('total_visualizacoes_dia', 'sum'),
        total_likes=('total_likes_dia', 'sum'),
        total_comentarios=('total_comentarios_dia', 'sum')
    ).reset_index()

In [21]:
df_duracao_video.head()

,data_extracao,nm_canal,total__visualizacoes,total_likes,total_comentarios
0,2023-10-15,ChratosGameplay,9124,1374,46
1,2023-10-16,ChratosGameplay,21502,3506,122
2,2023-10-19,Barbosa Cities,20649,4817,288
3,2023-10-19,ChratosGameplay,133688,21350,686
4,2023-10-20,Barbosa Cities,28906,3486,221


In [22]:
fig = px.bar(df_duracao_video, x='data_extracao', y='total__visualizacoes', barmode='group', color='nm_canal')
fig.show()

In [23]:
fig = px.bar(df_duracao_video, x='data_extracao', y='total_likes', barmode='group', color='nm_canal')
fig.show()

In [24]:
fig = px.bar(df_duracao_video, x='data_extracao', y='total_comentarios', barmode='group', color='nm_canal')
fig.show()

# Wordcloud TAGS

In [60]:
df_wordcloud_tags = pd.read_sql_query(
    ' SELECT tags '
    ' FROM total_visualizacoes_por_semana '
    ' WHERE assunto = "assunto_cities_skylines" ',
    con=engine
)
df_wordcloud_tags.head()

,tags
0,[]
1,"[""cities skylines"",""cities skylines gameplay"",..."
2,"[""xbae8vvXZp1LBYAKst0k"",""Machinimas"",""GAC"",""Ga..."
3,"[""euro truck 2"",""lucas gameplays brasil"",""mapa..."
4,"[""review"",""gameplay"",""jogo pc"",""jogos pc"",""pc ..."


In [152]:
df_wordcloud_tags_copy = df_wordcloud_tags.copy()

In [153]:
df_wordcloud_tags_copy.drop_duplicates(inplace=True)

In [154]:
df_wordcloud_tags_copy.head()

,tags
0,[]
1,"[""cities skylines"",""cities skylines gameplay"",..."
2,"[""xbae8vvXZp1LBYAKst0k"",""Machinimas"",""GAC"",""Ga..."
3,"[""euro truck 2"",""lucas gameplays brasil"",""mapa..."
4,"[""review"",""gameplay"",""jogo pc"",""jogos pc"",""pc ..."


In [155]:
teste = ["cities skylines","cities skylines gameplay"]

def partir_lista(teste):
     return ','.join(str(item) for item in teste)

partir_lista(teste=teste)

'cities skylines,cities skylines gameplay'

In [156]:
def converter_para_lista(texto):
    texto = texto.replace('[', '').replace(']', '').replace('"', '')
    lista = texto.split(', ')
    return lista


In [157]:
df_wordcloud_tags_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108 entries, 0 to 1446
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tags    108 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [158]:
df_wordcloud_tags_copy['tags'] = df_wordcloud_tags_copy['tags'].apply(converter_para_lista)

In [159]:
df_wordcloud_tags_copy.head()

,tags
0,[]
1,"[cities skylines,cities skylines gameplay,citi..."
2,"[xbae8vvXZp1LBYAKst0k,Machinimas,GAC,Gameplay,..."
3,"[euro truck 2,lucas gameplays brasil,mapa eaa,..."
4,"[review,gameplay,jogo pc,jogos pc,pc game,joga..."


In [160]:
df_wordcloud_tags_copy['tags_str'] = df_wordcloud_tags_copy['tags'].apply(partir_lista)
df_wordcloud_tags_copy.head()

,tags,tags_str
0,[],
1,"[cities skylines,cities skylines gameplay,citi...","cities skylines,cities skylines gameplay,citie..."
2,"[xbae8vvXZp1LBYAKst0k,Machinimas,GAC,Gameplay,...","xbae8vvXZp1LBYAKst0k,Machinimas,GAC,Gameplay,P..."
3,"[euro truck 2,lucas gameplays brasil,mapa eaa,...","euro truck 2,lucas gameplays brasil,mapa eaa,m..."
4,"[review,gameplay,jogo pc,jogos pc,pc game,joga...","review,gameplay,jogo pc,jogos pc,pc game,jogan..."


# Métricas especificas TRENDS

# Categrias Populares por data de extração

In [8]:
df_categoria_populares = pd.read_sql_query(
    ' SELECT data_extracao, NM_CANAL, TITULO_VIDEO,  ID_CATEGORIA, TOTAL_VISUALIZACOES_DIA, TOTAL_LIKES_DIA, TOTAL_COMENTARIOS_DIA'
    '  FROM trends_youtube '
    ' where data_extracao = "2023-10-15" ',
    con=engine
    )
df_categoria_populares.head()

,data_extracao,nm_canal,titulo_video,id_categoria,total_visualizacoes_dia,total_likes_dia,total_comentarios_dia
0,2023-10-15,東映アニメーション公式YouTubeチャンネル,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,1,4141018,171952.0,17758.0
1,2023-10-15,Paulinho e Toquinho Family,INCRÍVEL PISCINA de BOLINHAS DENTRO do CARRO,1,269891,3684.0,114.0
2,2023-10-15,A24,The Iron Claw | Official Trailer HD | A24,1,4279413,78174.0,5415.0
3,2023-10-15,Record TV Americas,Roberto Cabrini mostra a realidade das vítimas...,1,61574,NaN,140.0
4,2023-10-15,Vivziepop,Happy Day in Hell | Prime Video,1,1895056,233452.0,13365.0


In [9]:
df_categoria_populares.groupby(['data_extracao', 'id_categoria']) \
    .agg(
        total_visualizacoes=('total_visualizacoes_dia', 'sum'),
        total_likes = ('total_likes_dia', 'sum'),
        total_comentarios_dia=('total_comentarios_dia', 'sum')
    ).reset_index()

,data_extracao,id_categoria,total_visualizacoes,total_likes,total_comentarios_dia
0,2023-10-15,1,10646952,487262.0,36792.0
1,2023-10-15,10,188470120,8054410.0,505272.0
2,2023-10-15,17,25971018,942816.0,50166.0
3,2023-10-15,20,29314189,1747021.0,116548.0
4,2023-10-15,22,6772581,620345.0,49841.0
5,2023-10-15,23,1361139,68949.0,1486.0
6,2023-10-15,24,285267745,11743071.0,572240.0
7,2023-10-15,25,11520764,248596.0,13235.0
8,2023-10-15,26,49953,9076.0,690.0
9,2023-10-15,27,5215689,263761.0,12350.0


- Vídeo populares (likes, comentarios e visualizações)

In [11]:
df_categoria_populares.head()

,data_extracao,nm_canal,titulo_video,id_categoria,total_visualizacoes_dia,total_likes_dia,total_comentarios_dia
0,2023-10-15,東映アニメーション公式YouTubeチャンネル,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,1,4141018,171952.0,17758.0
1,2023-10-15,Paulinho e Toquinho Family,INCRÍVEL PISCINA de BOLINHAS DENTRO do CARRO,1,269891,3684.0,114.0
2,2023-10-15,A24,The Iron Claw | Official Trailer HD | A24,1,4279413,78174.0,5415.0
3,2023-10-15,Record TV Americas,Roberto Cabrini mostra a realidade das vítimas...,1,61574,NaN,140.0
4,2023-10-15,Vivziepop,Happy Day in Hell | Prime Video,1,1895056,233452.0,13365.0


- Canais populares (likes, comentarios e visualizações)

In [12]:
df_categoria_populares.groupby(['data_extracao', 'nm_canal']) \
    .agg(
        total_visualizacoes=('total_visualizacoes_dia', 'sum'),
        total_likes = ('total_likes_dia', 'sum'),
        total_comentarios_dia=('total_comentarios_dia', 'sum')
    ).reset_index()

,data_extracao,nm_canal,total_visualizacoes,total_likes,total_comentarios_dia
0,2023-10-15,(G)I-DLE (여자)아이들 (Official YouTube Channel),8627209,476803.0,19048.0
1,2023-10-15,30PRAUM,932037,116511.0,9296.0
2,2023-10-15,5incominutos,165650,27148.0,1994.0
3,2023-10-15,A Fazenda,1190662,25723.0,0.0
4,2023-10-15,A24,4279413,78174.0,5415.0
...,...,...,...,...,...
183,2023-10-15,ge,3921474,109948.0,9405.0
184,2023-10-15,k a m a i t a c h i,113624,28687.0,2104.0
185,2023-10-15,starshipTV,29838674,780081.0,45617.0
186,2023-10-15,viniccius13,1607585,259444.0,10361.0
